# Imports

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import cv2

import os

In [ ]:
X = pd.read_csv('/Users/antoine/Documents/DataScientest/Projet/data/X_train.csv', index_col=0)
y = pd.read_csv('/Users/antoine/Documents/DataScientest/Projet/data/Y_train.csv', index_col=0)

# Train / val / test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0, stratify=y)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0, stratify=y_train)

# Resize & sort images

In [ ]:
original_img_dir = "/Users/antoine/Pictures/DataScientest_Rakuten/original/image_train/"
train_set_path = "/Users/antoine/Pictures/DataScientest_Rakuten/sorted/train_dataset/"
val_set_path = "/Users/antoine/Pictures/DataScientest_Rakuten/sorted/val_dataset/"

In [ ]:
def img_preprocessing(img_filename, input_dir, output_dir, width=224, height=224, cropping=True):

    """
    - Par défaut, recadre l'image en lui retirant ses marges blanches si elle en a ;
    - Redimensionne l'image en 224 x 224 par défaut, ou selon les dimensions passées en arguments 'width' et 'height'
    """

    # Lecture de l'image avec OpenCV
    img = cv2.imread(input_dir + img_filename)

    if cropping:
        # Conversion de l'image en niveaux de gris
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
        # Inversion des niveaux de gris
        img_reversed = cv2.bitwise_not(img_gray)
    
        # Recherche des contours dans l'image
        contours, hierarchy = cv2.findContours(img_reversed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
        if contours:
            # Recherche du contour le plus externe
            contour_external = max(contours, key=cv2.contourArea)
    
            # Recherche des coordonnées du rectangle englobant le contour externe
            x_min, y_min, w, h = cv2.boundingRect(contour_external)
    
            # Rognement de l'image en utilisant les coordonnées du rectangle englobant
            img = img[y_min:y_min+h, x_min:x_min+w]

    # Redimensionnement de l'image
    img_resized = cv2.resize(img, (width, height))
    
    # Enregistrement de l'image dans le répertoire de sortie
    cv2.imwrite(os.path.join(output_dir + img_filename), img_resized)
    

In [ ]:
for X_, y_, path in zip((X_train, X_val), (y_train, y_val), (train_set_path, val_set_path)):

    for prdtypecode in y_['prdtypecode'].unique():
        if not os.path.exists(path + str(prdtypecode)):
            os.makedirs(path + str(prdtypecode))
            indexes = y_.loc[y_['prdtypecode']==prdtypecode].index
            for idx in indexes:
                filename = f"image_{X_.loc[idx, 'imageid']}_product_{X_.loc[idx, 'productid']}.jpg"
                img_preprocessing(filename, input_dir = original_img_dir, output_dir = path + str(prdtypecode) + '/')
